In [1]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import os

In [4]:
# Replace these with your Azure Storage account details
account_url = "https://tradetatics.blob.core.windows.net"
account_key = "ZkAJWrsEc1GVv7n2QzZvGsEx5B2hcBcy4Nk+QjNvvAtb+ntX/mCB1zzDdxlDLc1Bhy8vySPNqcjd+AStgvf4OQ=="
container_name = "daily-tech-analysis"

In [5]:
# Set up the BlobServiceClient
blob_service_client = BlobServiceClient(account_url=account_url, credential=account_key)


In [6]:
# Ensure the container exists
container_client = blob_service_client.get_container_client(container_name)
try:
    container_client.create_container()
except Exception as e:
    print(f"Container '{container_name}' already exists or cannot be created: {e}")

Container 'daily-tech-analysis' already exists or cannot be created: The specified container already exists.
RequestId:90bfc685-e01e-0046-0675-2f8c01000000
Time:2024-11-05T11:24:52.7721775Z
ErrorCode:ContainerAlreadyExists
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>ContainerAlreadyExists</Code><Message>The specified container already exists.
RequestId:90bfc685-e01e-0046-0675-2f8c01000000
Time:2024-11-05T11:24:52.7721775Z</Message></Error>


In [36]:
# Function to upload a file
def upload_file_to_blob(local_file_path, blob_name):
    blob_client = container_client.get_blob_client(blob_name)
    
    with open(local_file_path, "rb") as data:
        blob_client.upload_blob(data, overwrite=True)
    print(f"Uploaded '{local_file_path}' to blob '{blob_name}'")

# Function to download a file
def download_blob_to_file(blob_name, download_file_path):
    blob_client = container_client.get_blob_client(blob_name)
    
    with open(download_file_path, "wb") as download_file:
        download_file.write(blob_client.download_blob().readall())
    print(f"Downloaded blob '{blob_name}' to '{download_file_path}'")

In [11]:
# Example usage
local_upload_file = r"c:\Users\dipes\Desktop\stock_market\cron_job\my_database.db"
blob_name = "my_database.db"
download_file_path = r"c:\Users\dipes\Desktop\stock_market\cron_job\azure.db"

In [15]:
upload_file_to_blob(local_upload_file, blob_name)

Uploaded 'c:\Users\dipes\Desktop\stock_market\cron_job\my_database.db' to blob 'my_database.db'


In [12]:
download_blob_to_file(blob_name, download_file_path)

Downloaded blob 'my_database.db' to 'c:\Users\dipes\Desktop\stock_market\cron_job\azure.db'


In [5]:
import pandas as pd

data = {
  "calories": [420, 380, 390],
  "duration": [50, 40, 45]
}

#load data into a DataFrame object:
df = pd.DataFrame(data)

In [1]:
from datetime import datetime, date

In [2]:
def get_current_date_time():
    now = datetime.now()
    current_date = now.strftime("%Y-%m-%d")
    current_time = now.strftime("%H:%M:%S")
    return current_date, current_time

In [9]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import os, shutil

def account_details():
    account_url = "https://tradetatics.blob.core.windows.net"
    account_key = "ZkAJWrsEc1GVv7n2QzZvGsEx5B2hcBcy4Nk+QjNvvAtb+ntX/mCB1zzDdxlDLc1Bhy8vySPNqcjd+AStgvf4OQ=="
    container_name = "daily-tech-analysis"
    return account_url, account_key, container_name

# Function to upload a file
def upload_file_to_blob(local_file_path, blob_name):
    account_url, account_key, container_name = account_details()
    blob_service_client = BlobServiceClient(account_url=account_url, credential=account_key)
    container_client = blob_service_client.get_container_client(container_name)

    blob_client = container_client.get_blob_client(blob_name)
    
    with open(local_file_path, "rb") as data:
        blob_client.upload_blob(data, overwrite=True)
    print(f"Uploaded '{local_file_path}' to blob '{blob_name}'")

def save_data_blob(stk_df, tmp_loc='tmp'):
    current_date, current_time = get_current_date_time()
    file_name = f"Trend_Report_{current_date.replace('-','_')}.csv"
    if not os.path.exists(tmp_loc):
        os.mkdir(tmp_loc)
    tmp_file_path = os.path.join(tmp_loc, file_name)
    stk_df.to_csv(tmp_file_path, index=False)
    upload_file_to_blob(tmp_file_path, file_name)
    shutil.rmtree(tmp_loc)

In [12]:
save_data_blob(df)

Uploaded 'tmp\Trend_Report_2024_11_05.csv' to blob 'Trend_Report_2024_11_05.csv'


In [16]:
df

,calories,duration
0,420,50
1,380,40
2,390,45


In [10]:
account_url, account_key, container_name = account_details()
current_date, current_time = get_current_date_time()
file_name = f"Trend_Report_{current_date.replace('-','_')}.csv"

In [11]:
blob_service_client = BlobServiceClient(account_url=account_url, credential=account_key)
container_client = blob_service_client.get_container_client(container_name)
blob_client = container_client.get_blob_client(file_name)

In [13]:
blob_list = container_client.list_blobs()
for blob in blob_list:
    print(blob.name)

Trend_Report_2024_11_05.csv
Trend_Report_2024_11_06.csv


In [20]:
import re
f = [blob.name for blob in container_client.list_blobs()][-1]
match = re.search(r'(\d{4}_\d{2}_\d{2})', f)
date = match.group(1)
date

'2024_11_06'

In [10]:
downloaded_data = blob_client.download_blob().readall().decode("utf-8")
print(downloaded_data)

STOCK,COMPANY_NAME,MARKET_CAP(CR),VOLUME,RELATIVE_VOLUME,PRICE,TREND_THREE,TREND_SIX,RSI,RSI_DIFF,ATR,ATR_DIFF,RSI_INDICATOR,UPTREND_INDICATOR,MOVING_AVG_IND,STRENGTH,CHART_INDICATOR,BREAKOUT,LAST_THREE_CANDEL,NADARAYA_WATSON,VOL_PRC_CORR,PRICE_DIFF,PEICE_GAP_PCTG,CDL_NME_YES,CDL_SCR_YES,CDL_NME_TDY,CDL_SCR_TDY
20MICRONS,20 Microns Limited,507,33759.0,0.68,247.65,DECREASING,INCREASING,25.58,-1.61,9.64,3.66,BUY,SELL,SELL,0.44,STRONG SELL,False,True,STAY,0.29,0.0,-2.16,"['DOJI', 'GRAVESTONEDOJI', 'LONGLEGGEDDOJI']",300,['DOJI'],100
21STCENMGM,21st Century Management Services Limited,43,25319.0,1.86,119.06,INCREASING,INCREASING,59.6,3.35,2.35,-7.57,STAY,BUY,BUY,0.29,STAY,False,False,STAY,0.02,0.47,0.0,,0,,0
360ONE,360 ONE WAM LIMITED,24236,112874.0,0.36,1058.8,NO TREND,INCREASING,51.64,2.99,47.23,2.81,STAY,BUY,STRONG BUY,0.18,STAY,False,False,STAY,-0.19,2.22,0.0,"['DOJI', 'DOJISTAR', 'LONGLEGGEDDOJI', 'HIGHWAVE', 'SPINNINGTOP']",500,"['MORNINGDOJISTAR', 'MORNINGSTAR']",200
3IINFOLTD,3i In

In [6]:
from io import StringIO
import pandas as pd
# df = pd.read_csv(StringIO(downloaded_data))

# # Display the DataFrame
# df

In [21]:
def load_data():
    account_url, account_key, container_name = account_details()
    current_date, current_time = get_current_date_time()
    blob_service_client = BlobServiceClient(account_url=account_url, credential=account_key)
    container_client = blob_service_client.get_container_client(container_name)
    try:
        file_name = f"Trend_Report_{current_date.replace('-','_')}.csv"
        blob_client = container_client.get_blob_client(file_name)
        downloaded_data = blob_client.download_blob().readall().decode("utf-8")
        stk_df = pd.read_csv(StringIO(downloaded_data))
    except:
        file_name = [blob.name for blob in container_client.list_blobs()][-1]
        blob_client = container_client.get_blob_client(file_name)
        downloaded_data = blob_client.download_blob().readall().decode("utf-8")
        stk_df = pd.read_csv(StringIO(downloaded_data))
        
    match = re.search(r'(\d{4}_\d{2}_\d{2})', file_name)
    date = match.group(1)
    return stk_df, date

In [22]:
df, date = load_data()
df

,STOCK,COMPANY_NAME,MARKET_CAP(CR),VOLUME,RELATIVE_VOLUME,PRICE,TREND_THREE,TREND_SIX,RSI,RSI_DIFF,...,BREAKOUT,LAST_THREE_CANDEL,NADARAYA_WATSON,VOL_PRC_CORR,PRICE_DIFF,PEICE_GAP_PCTG,CDL_NME_YES,CDL_SCR_YES,CDL_NME_TDY,CDL_SCR_TDY
0,20MICRONS,20 Microns Limited,507,33759.0,0.68,247.65,DECREASING,INCREASING,25.58,-1.61,...,False,True,STAY,0.29,0.00,-2.16,"['DOJI', 'GRAVESTONEDOJI', 'LONGLEGGEDDOJI']",300,['DOJI'],100
1,21STCENMGM,21st Century Management Services Limited,43,25319.0,1.86,119.06,INCREASING,INCREASING,59.60,3.35,...,False,False,STAY,0.02,0.47,0.00,NaN,0,NaN,0
2,360ONE,360 ONE WAM LIMITED,24236,112874.0,0.36,1058.80,NO TREND,INCREASING,51.64,2.99,...,False,False,STAY,-0.19,2.22,0.00,"['DOJI', 'DOJISTAR', 'LONGLEGGEDDOJI', 'HIGHWA...",500,"['MORNINGDOJISTAR', 'MORNINGSTAR']",200
3,3IINFOLTD,3i Infotech Limited,667,496917.0,1.35,29.18,DECREASING,DECREASING,50.45,10.04,...,False,False,STAY,0.47,4.21,0.00,['SPINNINGTOP'],100,['MARUBOZU'],100
4,3MINDIA,3M India Limited,35139,1892.0,0.37,35385.35,NO TREND,INCREASING,54.86,-0.78,...,False,False,STAY,0.04,0.24,0.00,NaN,0,"['DOJI', 'GRAVESTONEDOJI', 'LONGLEGGEDDOJI', '...",400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2337,ZOTA,Zota Health Care LImited,1226,5626.0,0.90,611.90,DECREASING,INCREASING,49.05,-0.84,...,False,False,STAY,-0.31,-0.55,0.00,NaN,0,NaN,0
2338,ZUARI,Zuari Agro Chemicals Limited,712,156444.0,1.86,205.99,DECREASING,INCREASING,62.20,6.18,...,False,False,STAY,0.36,2.84,0.00,['SPINNINGTOP'],100,NaN,0
2339,ZUARIIND,ZUARI INDUSTRIES LIMITED,978,44845.0,0.67,341.50,DECREASING,INCREASING,47.09,0.58,...,False,False,STAY,0.40,-0.03,0.00,"['DOJI', 'DRAGONFLYDOJI', 'LONGLEGGEDDOJI']",100,"['DOJI', 'GRAVESTONEDOJI', 'LONGLEGGEDDOJI']",0
2340,ZYDUSLIFE,Zydus Lifesciences Limited,101964,616297.0,0.69,1003.00,DECREASING,INCREASING,43.61,6.30,...,False,False,STAY,0.26,1.47,0.00,NaN,0,NaN,0


In [23]:
date

'2024_11_06'

In [1]:
import pandas as pd

df = pd.read_csv('tmp/Trend_Report_2024_11_07.csv')
df.head()

,STOCK,COMPANY_NAME,MARKET_CAP(CR),VOLUME,RELATIVE_VOLUME,PRICE,TREND_THREE,TREND_SIX,RSI,RSI_DIFF,...,BREAKOUT,LAST_THREE_CANDEL,NADARAYA_WATSON,VOL_PRC_CORR,PRICE_DIFF,PEICE_GAP_PCTG,CDL_NME_YES,CDL_SCR_YES,CDL_NME_TDY,CDL_SCR_TDY
0,20MICRONS,20 Microns Limited,507,25234.0,0.79,242.70,DECREASING,INCREASING,24.98,-1.58,...,False,True,STAY,0.22,0.00,-2.04,['DOJI'],100,['DOJI'],100
1,21STCENMGM,21st Century Management Services Limited,43,8119.0,0.53,121.44,INCREASING,INCREASING,63.97,3.04,...,False,False,STAY,0.02,0.20,1.60,NaN,0,NaN,0
2,360ONE,360 ONE WAM LIMITED,24236,110840.0,0.44,1040.60,NO TREND,INCREASING,47.37,-2.11,...,False,False,STAY,-0.18,-1.87,0.00,"['MORNINGDOJISTAR', 'MORNINGSTAR']",200,NaN,-100
3,3IINFOLTD,3i Infotech Limited,667,419382.0,1.06,29.32,DECREASING,DECREASING,48.80,0.28,...,False,False,STAY,0.49,-1.28,0.00,NaN,0,NaN,0
4,3MINDIA,3M India Limited,35139,2978.0,0.82,36052.40,NO TREND,INCREASING,58.81,4.97,...,False,False,STAY,0.01,0.65,0.00,['INVERTEDHAMMER'],100,['SPINNINGTOP'],100


In [2]:
d = df[['STOCK','PRICE', 'PRICE_DIFF']].sort_values(by='PRICE_DIFF', ascending= False).reset_index(drop=True)

In [3]:
txt = '", "'.join(list(d[:1].astype(str).values[0]))
txt = txt.replace("'", '"')
txt

'HERCULES", "239.61", "20.41'

In [4]:
g = [list(v.values) for i, v in d.iterrows() if v.values[2] > 5][:16]

In [5]:
round(g[0][1])

240

In [6]:
g

[['HERCULES', 239.61, 20.41],
 ['LOKESHMACH', 393.6, 18.23],
 ['SWANENERGY', 566.95, 14.78],
 ['PYRAMID', 254.49, 14.14],
 ['MADHUSUDAN', 207.25, 13.87],
 ['TERASOFT', 98.09, 13.01],
 ['KIOCL', 386.0, 12.54],
 ['DENEERS', 252.8, 12.36],
 ['SERVICE', 87.5, 12.18],
 ['PATINTLOG', 24.2, 11.06],
 ['ROXHITECH', 124.25, 10.0],
 ['PENINLAND', 54.1, 9.96],
 ['NECCLTD', 36.67, 9.79],
 ['PROLIFE', 275.1, 9.49],
 ['KEC', 1065.3, 9.15],
 ['ITI', 256.94, 9.1]]

In [7]:
l = [list(v.values) for i, v in d.iterrows() if v.values[2] < -5][::-1]

In [8]:
l

[['GRETEX', 264.0, -8.98],
 ['CREATIVEYE', 8.17, -8.72],
 ['DHTL', 82.05, -8.07],
 ['TRENT', 6454.6, -7.78],
 ['KEL', 140.1, -7.22],
 ['MORARJEE', 9.91, -7.12],
 ['OWAIS', 1391.4, -7.05],
 ['BLUESTARCO', 1782.0, -7.0],
 ['NITIRAJ', 241.99, -6.96],
 ['JETAIRWAYS', 34.16, -6.9],
 ['TFL', 26.81, -6.75],
 ['SVLL', 335.0, -6.69],
 ['STEELCITY', 110.48, -6.37],
 ['EMKAYTOOLS', 1386.15, -6.34],
 ['SAMPANN', 32.76, -6.27],
 ['SOMATEX', 46.55, -6.24],
 ['PRAMARA', 150.2, -6.07],
 ['ESTER', 164.43, -6.04],
 ['AUSOMENT', 107.55, -5.99],
 ['VILINBIO', 23.7, -5.95],
 ['MODTHREAD', 57.8, -5.94],
 ['ENERGYDEV', 24.26, -5.93],
 ['PILITA', 15.4, -5.81],
 ['GLENMARK', 1668.45, -5.68],
 ['FCSSOFT', 3.65, -5.68],
 ['VENKEYS', 1740.85, -5.64],
 ['HOMESFY', 506.0, -5.6],
 ['NITINSPIN', 409.05, -5.52],
 ['ELGIRUBCO', 124.74, -5.5],
 ['JAINAM', 166.0, -5.41],
 ['NDGL', 5430.0, -5.4],
 ['JUBLINDS', 1633.5, -5.3],
 ['KPIGREEN', 794.55, -5.18],
 ['CANARYS', 41.6, -5.13],
 ['TIL', 336.0, -5.07],
 ['INNOVANA', 465

In [12]:
df.columns

Index(['STOCK', 'COMPANY_NAME', 'MARKET_CAP(CR)', 'VOLUME', 'RELATIVE_VOLUME',
       'PRICE', 'TREND_THREE', 'TREND_SIX', 'RSI', 'RSI_DIFF', 'ATR',
       'ATR_DIFF', 'RSI_INDICATOR', 'UPTREND_INDICATOR', 'MOVING_AVG_IND',
       'STRENGTH', 'CHART_INDICATOR', 'BREAKOUT', 'LAST_THREE_CANDEL',
       'NADARAYA_WATSON', 'VOL_PRC_CORR', 'PRICE_DIFF', 'PEICE_GAP_PCTG',
       'CDL_NME_YES', 'CDL_SCR_YES', 'CDL_NME_TDY', 'CDL_SCR_TDY'],
      dtype='object')

In [10]:
data = df

In [18]:
brkout_high_vol = data[(data['BREAKOUT'] == True)& (data['PRICE_DIFF'] > 0)].sort_values('RELATIVE_VOLUME',ascending=False).reset_index(drop=True)
brkout_high_vol = brkout_high_vol[['STOCK','PRICE', 'PRICE_DIFF']]
brkout_high_vol = [list(v.values) for i, v in brkout_high_vol.iterrows()][:14]
brkout_high_vol

[['DENEERS', 252.8, 12.36],
 ['AVALON', 694.8, 4.48],
 ['AATMAJ', 28.0, 7.69],
 ['TERASOFT', 98.09, 13.01],
 ['TEMBO', 662.85, 1.35],
 ['SIDDHIKA', 141.3, 0.21],
 ['ROXHITECH', 124.25, 10.0],
 ['PENINLAND', 54.1, 9.96],
 ['MADHUSUDAN', 207.25, 13.87],
 ['SIGNORIA', 159.1, 7.39],
 ['APOLLOHOSP', 7440.35, 1.69],
 ['MDL', 115.2, 6.86],
 ['ICEMAKE', 920.5, 6.54],
 ['CREATIVE', 983.05, 5.6]]

In [19]:
def stream_matrix(gainers, header):
    st.header(header)
    g0, g1, g2, g3, g4, g5, g6= st.columns(7)
    g0.metric(gainers[0][0], round(gainers[0][1]), str(round(gainers[0][2])) + '%')
    g1.metric(gainers[1][0], round(gainers[1][1]), str(round(gainers[1][2])) + '%')
    g2.metric(gainers[2][0], round(gainers[2][1]), str(round(gainers[2][2])) + '%')
    g3.metric(gainers[3][0], round(gainers[3][1]), str(round(gainers[3][2])) + '%')
    g4.metric(gainers[4][0], round(gainers[4][1]), str(round(gainers[4][2])) + '%')
    g5.metric(gainers[5][0], round(gainers[5][1]), str(round(gainers[5][2])) + '%')
    g6.metric(gainers[6][0], round(gainers[6][1]), str(round(gainers[6][2])) + '%')
    

    g7, g8, g9, g10, g11, g12, g13= st.columns(7)
    g7.metric(gainers[7][0], round(gainers[7][1]), str(round(gainers[7][2])) + '%')
    g8.metric(gainers[8][0], round(gainers[8][1]), str(round(gainers[8][2])) + '%')
    g9.metric(gainers[9][0], round(gainers[9][1]), str(round(gainers[9][2])) + '%')
    g10.metric(gainers[10][0], round(gainers[10][1]), str(round(gainers[10][2])) + '%')
    g11.metric(gainers[11][0], round(gainers[11][1]), str(round(gainers[11][2])) + '%')
    g12.metric(gainers[12][0], round(gainers[12][1]), str(round(gainers[12][2])) + '%')
    g13.metric(gainers[13][0], round(gainers[13][1]), str(round(gainers[13][2])) + '%')
            
stream_matrix(brkout_high_vol)            

0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
2 13


In [ ]:
gainers_losers_df= data[['STOCK','PRICE', 'PRICE_DIFF']].sort_values(by='PRICE_DIFF', ascending= False).reset_index(drop=True)
    st.header('Top Gainers')
    gainers = [list(v.values) for i, v in gainers_losers_df.iterrows() if v.values[2] > 5]
    g0, g1, g2, g3, g4, g5, g6= st.columns(7)
    g0.metric(gainers[0][0], round(gainers[0][1]), str(round(gainers[0][2])) + '%')
    g1.metric(gainers[1][0], round(gainers[1][1]), str(round(gainers[1][2])) + '%')
    g2.metric(gainers[2][0], round(gainers[2][1]), str(round(gainers[2][2])) + '%')
    g3.metric(gainers[3][0], round(gainers[3][1]), str(round(gainers[3][2])) + '%')
    g4.metric(gainers[4][0], round(gainers[4][1]), str(round(gainers[4][2])) + '%')
    g5.metric(gainers[5][0], round(gainers[5][1]), str(round(gainers[5][2])) + '%')
    g6.metric(gainers[6][0], round(gainers[6][1]), str(round(gainers[6][2])) + '%')
    

    g7, g8, g9, g10, g11, g12, g13= st.columns(7)
    g7.metric(gainers[7][0], round(gainers[7][1]), str(round(gainers[7][2])) + '%')
    g8.metric(gainers[8][0], round(gainers[8][1]), str(round(gainers[8][2])) + '%')
    g9.metric(gainers[9][0], round(gainers[9][1]), str(round(gainers[9][2])) + '%')
    g10.metric(gainers[10][0], round(gainers[10][1]), str(round(gainers[10][2])) + '%')
    g11.metric(gainers[11][0], round(gainers[11][1]), str(round(gainers[11][2])) + '%')
    g12.metric(gainers[12][0], round(gainers[12][1]), str(round(gainers[12][2])) + '%')
    g13.metric(gainers[13][0], round(gainers[13][1]), str(round(gainers[13][2])) + '%')